In [1]:
from suffix_trees import STree
import suffix_tree
import pandas as pd

In [2]:
class STreeEx():
    def __init__(self, sTree):
        # 傳入原套件後綴樹
        self.sTree = sTree

    def lrs(self):
        # 最深非葉節點
        deepestNode = self._find_lrs(self.sTree.root)
        start = deepestNode.idx
        end = deepestNode.idx + deepestNode.depth
        return self.sTree.word[start:end]

    def _find_lrs(self, node):
        nodes = [self._find_lrs(n)
            for (n,_) in node.transition_links
            # 排除葉節點
            if n.transition_links != []]

        if nodes == []:
            return node
        
        deepestNode = max(nodes, key=lambda n: n.depth)
        return deepestNode

# Input File

In [3]:
input_file_path = "./input/TableA.txt"

In [4]:
content = []
recb_start = 0
recb_end = 0
tag = []
ids = []
classes = []
pathid = []
parentid = []
tecid = []
cecid = []
encoding = []
col = []
others = []
with open(input_file_path) as file:
    line = file.readline()
    while(line):
        line = file.readline()
        tmp = line.split('\t')
        if tmp[0] == "Content":
            content = tmp[1:]
        elif tmp[0] == "RecB":
            tmp = tmp[1:]
            for index in range(len(tmp)):
                if tmp[index] == "SDR":
                    recb_start = index
                elif tmp[index] == "EDR":
                    recb_end = index
        elif tmp[0] == "Tag":
            tag = tmp[1:]
        elif tmp[0] == "ID":
            ids = tmp[1:]
        elif tmp[0] == "Class":
            classes = tmp[1:]
        elif tmp[0] == "PathId":
            pathid = tmp[1:]
        elif tmp[0] == "ParentId":
            parentid = tmp[1:]
        elif tmp[0] == "TECId":
            tecid = tmp[1:]
        elif tmp[0] == "CECId":
            cecid = tmp[1:]
        elif tmp[0] == "Encoding":
            encoding = tmp[1:]
        elif tmp[0] == "Col Type":
            col = tmp[1:]
        else:
            others.append(tmp[1:])
others = others[0:-4]

In [5]:
tec_dict = {}
unique_mt = []
for node in range(len(col)):
    if col[node] == 'MT':
        if tecid[node] not in tec_dict.keys():
            tec_dict[tecid[node]] = [node]
        else:
            tec_dict[tecid[node]].append(node)
for key in tec_dict.keys():
    if len(tec_dict[key]) == 1:
        unique_mt += tec_dict[key]
print("Unique MT's index:\n", unique_mt)

Unique MT's index:
 [3, 4, 5, 6, 8, 10, 16, 18, 19, 20, 22, 23, 25, 28, 81, 136, 279, 281, 284, 291, 303, 306, 308]


# Translate node in to unicode for suffix tree

Change `encode` to use different attribute to encode leafnodes.
- 0: **Pathid** + **Parentid** + **TECid** + **CECid** + **Encoding** + **Column Type**
- 1: **Id** + **Class** + **Pathid** + **CECid**
- 2: **Pathid** + **Parentid** + **CECid**
<br><br>
Leafnodes will have same encoding if they have same attribute.

In [6]:
encode = 2
whole_string = ""
test = ""
node_dict = {} # code -> node num
code_dict = {} # code -> content
index_dict = {} # code -> first index
index = 0
for node in range(len(pathid)):
    if encode == 0:
        code = pathid[node] + "," + parentid[node] + "," + tecid[node] + "," + cecid[node] + "," + encoding[node] + "," + col[node]
    elif encode == 1:
        code = ids[node] + "," + classes[node] + "," + pathid[node] + "," + cecid[node]
    elif encode == 2:
        code = pathid[node] + "," + parentid[node] + "," + cecid[node]
    #if 'O' in col[node]: continue
    if code not in node_dict.keys():
        node_dict[code] = index
        code_dict[code] = others[0][node]
        index_dict[code] = node
        whole_string += chr(index)
        index += 1
        if index == 45 or index == 32:
            index+=1
    else:
        whole_string += chr(node_dict[code])
print("Example: ")
if encode == 0:
    print(pathid[0] + "," + parentid[0] + "," + tecid[0] + "," + cecid[0] + "," + encoding[0] + "," + col[0])
elif encode == 1:
    print(ids[0] + "," + classes[0] + "," + pathid[0] + "," + cecid[0])
if encode == 2:
    print(pathid[0] + "," + parentid[0] + "," + cecid[0])
inv_node_dict = {v: k for k, v in node_dict.items()} # node num -> code
inv_index_dict = {v: k for k, v in index_dict.items()} # index num -> code
print("\nConvert to Unicode String:\n", whole_string)

Example: 
0,"1:[1, 2]", 

Convert to Unicode String:
  	
!"#$%&'()*!"#$%&'()*+,./.0.1.2.3.4.5.6.7.8.9.:.;.<.=.>.?.@.A.B.C.D.E.F.G.H.I.JJKLLMKNKOPJJKLLQNKOKJJKLLMKNKOKJJKLLMKNKOPJJKLLMQNKOKJJKLLMKNKOKJJKLLMKNKOKJJKLLMKNKOKJJKLLKNKOPJJKLLMKNKOPJJKLLMQNKOKJJKLLKNKOKRSTUVWXYZ[\]^_`abcdefghijklmnopqrstu


In [7]:
segments = []
for seg_point in range(len(unique_mt)):
    if seg_point != len(unique_mt)-1:
        tmp = whole_string[unique_mt[seg_point]+1:unique_mt[seg_point+1]]
        if len(tmp) > 10:
            segments.append(tmp)
segments = list(filter(None, segments))
print(len(segments))
print(segments)

4
['\x1c\x1d\x1e\x1d\x1f\x1d!\x1d"\x1d#\x1d$\x1d%\x1d&\x1d\'\x1d(\x1d)\x1d*\x1d\x1c\x1d\x1e\x1d\x1f\x1d!\x1d"\x1d#\x1d$\x1d%\x1d&\x1d\'\x1d(\x1d)\x1d*\x1d', ',./.0.1.2.3.4.5.6.7.8.9.:.;.<.=.>.?.@.A.B.C.D.E.F.G.H.', '.JJKLLMKNKOPJJKLLQNKOKJJKLLMKNKOKJJKLLMKNKOPJJKLLMQNKOKJJKLLMKNKOKJJKLLMKNKOKJJKLLMKNKOKJJKLLKNKOPJJKLLMKNKOPJJKLLMQNKOKJJKLLKNKOKR\x0cS\x0cT\x0cU\x0cV\x0cW\x0c', 'defghijklmn']


In [8]:
def find_all_indexes(input_str, search_str):
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1

# Longest repeat pattern

In [9]:
"""st = STree.STree(whole_string)
st = STreeEx(st)
lrs = str(st.lrs())
print("Length: ", len(lrs))
print(lrs)
for i in lrs:
    print("Content: ", code_dict[inv_node_dict[ord(i)]])"""

'st = STree.STree(whole_string)\nst = STreeEx(st)\nlrs = str(st.lrs())\nprint("Length: ", len(lrs))\nprint(lrs)\nfor i in lrs:\n    print("Content: ", code_dict[inv_node_dict[ord(i)]])'

# Show maximal repeats

Ignore maximal repeats if length equal to 1.

In [10]:
"""tree = suffix_tree.Tree({'A': whole_string})
print("Total Maximal repeats: ", len(tree.maximal_repeats()), '\n')
group = {}
top_repeats = (0, '')
for C, path in tree.maximal_repeats():
    if len(path) == 1: continue
    count = 0
    print("Length: ", len(path), "Pattern: {", path, "}")
    for id_, path2 in tree.find_all (path):
        count += 1
    print("Count: ", count)
    if count > top_repeats[0]:
        top_repeats = (count, str(path).replace(' ', ''))
    for i in path:
        print("Content: ", code_dict[inv_node_dict[ord(i)]])
    if index_dict[inv_node_dict[ord(path[0])]] not in group.keys():
        group[index_dict[inv_node_dict[ord(path[0])]]] = []
        group[index_dict[inv_node_dict[ord(path[0])]]].append(path)
    else:
        group[index_dict[inv_node_dict[ord(path[0])]]].append(path)
    print("\n", "="*50, "\n")"""

'tree = suffix_tree.Tree({\'A\': whole_string})\nprint("Total Maximal repeats: ", len(tree.maximal_repeats()), \'\n\')\ngroup = {}\ntop_repeats = (0, \'\')\nfor C, path in tree.maximal_repeats():\n    if len(path) == 1: continue\n    count = 0\n    print("Length: ", len(path), "Pattern: {", path, "}")\n    for id_, path2 in tree.find_all (path):\n        count += 1\n    print("Count: ", count)\n    if count > top_repeats[0]:\n        top_repeats = (count, str(path).replace(\' \', \'\'))\n    for i in path:\n        print("Content: ", code_dict[inv_node_dict[ord(i)]])\n    if index_dict[inv_node_dict[ord(path[0])]] not in group.keys():\n        group[index_dict[inv_node_dict[ord(path[0])]]] = []\n        group[index_dict[inv_node_dict[ord(path[0])]]].append(path)\n    else:\n        group[index_dict[inv_node_dict[ord(path[0])]]].append(path)\n    print("\n", "="*50, "\n")'

In [11]:
record_seg = []
trees = [suffix_tree.Tree({'A': seg}) for seg in segments]
for tree_idx in range(len(trees)):
    tree = trees[tree_idx]
    #print("Total Maximal repeats: ", len(tree.maximal_repeats()), '\n')
    group = {}
    top_repeats = (0, '')
    for C, path in tree.maximal_repeats():
        if len(path) == 1: continue
        count = 0
        #print("Length: ", len(path), "Pattern: {", path, "}")
        for id_, path2 in tree.find_all (path):
            count += 1
        #print("Count: ", count)
        if count > top_repeats[0]:
            top_repeats = (count, str(path).replace(' ', ''))
        """for i in path:
            print("Content: ", code_dict[inv_node_dict[ord(i)]])"""
        if index_dict[inv_node_dict[ord(path[0])]] not in group.keys():
            group[index_dict[inv_node_dict[ord(path[0])]]] = []
            group[index_dict[inv_node_dict[ord(path[0])]]].append(path)
        else:
            group[index_dict[inv_node_dict[ord(path[0])]]].append(path)
        #print("\n", "="*50, "\n")
    if top_repeats[0] != 0:
        record_seg.append((tree_idx, top_repeats[1]))
        print(top_repeats)

(2, '\x1c\x1d\x1e\x1d\x1f\x1d!\x1d"\x1d#\x1d$\x1d%\x1d&\x1d\'\x1d(\x1d)\x1d*\x1d')
(12, 'JJKLL')


# Whole String split by top repeats

In [12]:
"""max_len = 0
seqs = []
pos = find_all_indexes(whole_string, top_repeats[1])
for idx in range(len(pos)):
    if idx+1 != len(pos):
        seqs.append(whole_string[pos[idx]:pos[idx+1]])
        if len(whole_string[pos[idx]:pos[idx+1]]) > max_len:
            max_len = len(whole_string[pos[idx]:pos[idx+1]])
    else:
        seqs.append(whole_string[pos[idx]:pos[idx]+max_len+4])
print(str(seqs))"""

'max_len = 0\nseqs = []\npos = find_all_indexes(whole_string, top_repeats[1])\nfor idx in range(len(pos)):\n    if idx+1 != len(pos):\n        seqs.append(whole_string[pos[idx]:pos[idx+1]])\n        if len(whole_string[pos[idx]:pos[idx+1]]) > max_len:\n            max_len = len(whole_string[pos[idx]:pos[idx+1]])\n    else:\n        seqs.append(whole_string[pos[idx]:pos[idx]+max_len+4])\nprint(str(seqs))'

In [13]:
max_len = []
all_seqs = []
for seg_idx in range(len(record_seg)):
    max_len.append(0)
    seqs = []
    pos = find_all_indexes(segments[record_seg[seg_idx][0]], record_seg[seg_idx][1])
    for idx in range(len(pos)):
        if idx+1 != len(pos):
            seqs.append(segments[record_seg[seg_idx][0]][pos[idx]:pos[idx+1]])
            if len(segments[record_seg[seg_idx][0]][pos[idx]:pos[idx+1]]) > max_len[seg_idx]:
                max_len[seg_idx] = len(segments[record_seg[seg_idx][0]][pos[idx]:pos[idx+1]])
        else:
            seqs.append(segments[record_seg[seg_idx][0]][pos[idx]:pos[idx]+max_len[seg_idx]+4])
    all_seqs.append(seqs)
print(all_seqs)

[['\x1c\x1d\x1e\x1d\x1f\x1d!\x1d"\x1d#\x1d$\x1d%\x1d&\x1d\'\x1d(\x1d)\x1d*\x1d', '\x1c\x1d\x1e\x1d\x1f\x1d!\x1d"\x1d#\x1d$\x1d%\x1d&\x1d\'\x1d(\x1d)\x1d*\x1d'], ['JJKLLMKNKOP', 'JJKLLQNKOK', 'JJKLLMKNKOK', 'JJKLLMKNKOP', 'JJKLLMQNKOK', 'JJKLLMKNKOK', 'JJKLLMKNKOK', 'JJKLLMKNKOK', 'JJKLLKNKOP', 'JJKLLMKNKOP', 'JJKLLMQNKOK', 'JJKLLKNKOKR\x0cS\x0cT']]


# MSA

In [14]:
"""import cstar
print("First round MSA\n", "="*100)
scores = [1, -1, -3] # matchScore, mismatchScore, gapScore
msa = cstar.CenterStar(scores, seqs[:-1]).msa()
trans_dict = {}
last_c = '-'
end_idx = 0

for i in range(len(msa)):
    if msa[i][-1] != '-' and last_c not in trans_dict.keys():
        last_c = msa[i][-1]
        trans_dict[last_c] = msa[i].replace('-', '')
    if msa[i].replace('-', '') not in trans_dict.keys():
        trans_dict[msa[i].replace('-', '')] = msa[i]
    else: pass
for i in seqs[:-1]:
    print(i, "\t-> ", trans_dict[i])

print('='*100, "\nSecond round MSA\n")
msa_2 = cstar.CenterStar(scores, msa+[seqs[-1]]).msa()
trans_dict_2 = {}

for i in range(len(msa_2)):
    if msa_2[i].replace('-', '') not in trans_dict_2.keys():
        trans_dict_2[msa_2[i].replace('-', '')] = msa_2[i]
    else: pass
for idx in range(len(trans_dict_2[trans_dict[last_c]])):
    if trans_dict_2[trans_dict[last_c]][idx] == last_c:
        end_idx = idx
for i in seqs:
    trans_dict_2[i] = trans_dict_2[i][:end_idx+1]
    print(i, "\t-> ", trans_dict_2[i])

with open('./Set.txt', 'w', encoding='utf-8') as file:
    for page in range(len(others)):
        output_dict = {} # Record which pattern is used
        for s in range(len(seqs)):
            file.write(str(page) + "-0" + "-" + str(s) + "\t")
            tmp = find_all_indexes(whole_string, seqs[s])
            if seqs[s] not in output_dict.keys():
                output_dict[seqs[s]] = 0
            else:
                output_dict[seqs[s]] += 1
            #print("start:", tmp[output_dict[seqs[s]]], others[page][tmp[output_dict[seqs[s]]]])
            idx = 0
            for c in range(len(trans_dict_2[seqs[s]])):
                if trans_dict_2[seqs[s]][c] == '-':
                    file.write('\t')
                else:
                    file.write(others[page][tmp[output_dict[seqs[s]]]+idx] + "\t")
                    idx += 1
            file.write('\n')"""

'import cstar\nprint("First round MSA\n", "="*100)\nscores = [1, -1, -3] # matchScore, mismatchScore, gapScore\nmsa = cstar.CenterStar(scores, seqs[:-1]).msa()\ntrans_dict = {}\nlast_c = \'-\'\nend_idx = 0\n\nfor i in range(len(msa)):\n    if msa[i][-1] != \'-\' and last_c not in trans_dict.keys():\n        last_c = msa[i][-1]\n        trans_dict[last_c] = msa[i].replace(\'-\', \'\')\n    if msa[i].replace(\'-\', \'\') not in trans_dict.keys():\n        trans_dict[msa[i].replace(\'-\', \'\')] = msa[i]\n    else: pass\nfor i in seqs[:-1]:\n    print(i, "\t-> ", trans_dict[i])\n\nprint(\'=\'*100, "\nSecond round MSA\n")\nmsa_2 = cstar.CenterStar(scores, msa+[seqs[-1]]).msa()\ntrans_dict_2 = {}\n\nfor i in range(len(msa_2)):\n    if msa_2[i].replace(\'-\', \'\') not in trans_dict_2.keys():\n        trans_dict_2[msa_2[i].replace(\'-\', \'\')] = msa_2[i]\n    else: pass\nfor idx in range(len(trans_dict_2[trans_dict[last_c]])):\n    if trans_dict_2[trans_dict[last_c]][idx] == last_c:\n      

In [15]:
import cstar
for seg_idx in range(len(all_seqs)):
    print("First round MSA\n", "="*100)
    scores = [1, -1, -3] # matchScore, mismatchScore, gapScore
    if len(all_seqs[seg_idx][:-1]) == 1:
        msa = all_seqs[seg_idx][:-1]
    else:
        msa = cstar.CenterStar(scores, all_seqs[seg_idx][:-1]).msa()
    trans_dict = {}
    last_c = '-'
    end_idx = 0

    for i in range(len(msa)):
        if msa[i][-1] != '-' and last_c not in trans_dict.keys():
            last_c = msa[i][-1]
            trans_dict[last_c] = msa[i].replace('-', '')
        if msa[i].replace('-', '') not in trans_dict.keys():
            trans_dict[msa[i].replace('-', '')] = msa[i]
        else: pass
    for i in all_seqs[seg_idx][:-1]:
        print(i, "\t-> ", trans_dict[i])

    print('='*100, "\nSecond round MSA\n")
    msa_2 = cstar.CenterStar(scores, msa+[all_seqs[seg_idx][-1]]).msa()
    trans_dict_2 = {}

    for i in range(len(msa_2)):
        if msa_2[i].replace('-', '') not in trans_dict_2.keys():
            trans_dict_2[msa_2[i].replace('-', '')] = msa_2[i]
        else: pass
    for idx in range(len(trans_dict_2[trans_dict[last_c]])):
        if trans_dict_2[trans_dict[last_c]][idx] == last_c:
            end_idx = idx
    for i in all_seqs[seg_idx]:
        trans_dict_2[i] = trans_dict_2[i][:end_idx+1]
        print(i, "\t-> ", trans_dict_2[i])

    with open('./Set-' + str(seg_idx) + '.txt', 'w', encoding='utf-8') as file:
        for page in range(len(others)):
            output_dict = {} # Record which pattern is used
            for s in range(len(all_seqs[seg_idx])):
                file.write(str(page) + "-" + str(seg_idx) + "-" + str(s) + "\t")
                tmp = find_all_indexes(whole_string, all_seqs[seg_idx][s])
                if all_seqs[seg_idx][s] not in output_dict.keys():
                    output_dict[all_seqs[seg_idx][s]] = 0
                else:
                    output_dict[all_seqs[seg_idx][s]] += 1
                #print("start:", tmp[output_dict[seqs[s]]], others[page][tmp[output_dict[seqs[s]]]])
                idx = 0
                for c in range(len(trans_dict_2[all_seqs[seg_idx][s]])):
                    if trans_dict_2[all_seqs[seg_idx][s]][c] == '-':
                        file.write('\t')
                    else:
                        file.write(others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx] + "\t")
                        idx += 1
                file.write('\n')

First round MSA
!"#$%&'()* 	->  !"#$%&'()*
Second round MSA

!"#$%&'()* 	->  !"#$%&'()*
!"#$%&'()* 	->  !"#$%&'()*
First round MSA
JJKLLMKNKOP 	->  JJKLLMKNKOP
JJKLLQNKOK 	->  JJKLL-QNKOK
JJKLLMKNKOK 	->  JJKLLMKNKOK
JJKLLMKNKOP 	->  JJKLLMKNKOP
JJKLLMQNKOK 	->  JJKLLMQNKOK
JJKLLMKNKOK 	->  JJKLLMKNKOK
JJKLLMKNKOK 	->  JJKLLMKNKOK
JJKLLMKNKOK 	->  JJKLLMKNKOK
JJKLLKNKOP 	->  JJKLL-KNKOP
JJKLLMKNKOP 	->  JJKLLMKNKOP
JJKLLMQNKOK 	->  JJKLLMQNKOK
Second round MSA

JJKLLMKNKOP 	->  JJKLLMKNKOP
JJKLLQNKOK 	->  JJKLL-QNKOK
JJKLLMKNKOK 	->  JJKLLMKNKOK
JJKLLMKNKOP 	->  JJKLLMKNKOP
JJKLLMQNKOK 	->  JJKLLMQNKOK
JJKLLMKNKOK 	->  JJKLLMKNKOK
JJKLLMKNKOK 	->  JJKLLMKNKOK
JJKLLMKNKOK 	->  JJKLLMKNKOK
JJKLLKNKOP 	->  JJKLL-KNKOP
JJKLLMKNKOP 	->  JJKLLMKNKOP
JJKLLMQNKOK 	->  JJKLLMQNKOK
JJKLLKNKOKRST 	->  JJKLL-KNKOK
